In [ ]:
# 0. IMPORTANDO AS BIBLIOTECAS NECESSÁRIAS

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score


In [ ]:
# 1. CARREGAMENTO

df = pd.read_csv('hour.csv')


In [ ]:
# 1.1 PRÉ-VISUALIZAÇÃO E ANÁLISE EXPLORATÓRIA (EDA)

print("\n" + "="*60)
print("INFORMAÇÕES GERAIS DO DATASET")
print("="*60)

# Dimensão
print(f"Dimensão do dataset: {df.shape[0]} linhas x {df.shape[1]} colunas\n")

# Primeiras linhas
print("Primeiras 5 linhas:")
print(df.head(), "\n")

# Tipos de dados
print("Tipos de variáveis:")
print(df.dtypes, "\n")

# Valores ausentes
print("Valores ausentes por coluna:")
print(df.isnull().sum(), "\n")

# Estatísticas descritivas
print("Estatísticas descritivas:")
print(df.describe(), "\n")


# Correlação com variável alvo

print("="*60)
print("CORRELAÇÃO COM A VARIÁVEL ALVO (cnt)")
print("="*60)

correlation = df.corr(numeric_only=True)['cnt'].sort_values(ascending=False)
print(correlation)


# VISUALIZAÇÕES EXPLORATÓRIAS

plt.figure(figsize=(18, 5))

# Distribuição do target
plt.subplot(1, 3, 1)
sns.histplot(df['cnt'], bins=50, kde=True)
plt.title("Distribuição do Total de Aluguéis (cnt)")
plt.xlabel("Quantidade de Bicicletas")
plt.ylabel("Frequência")

# Média por hora
plt.subplot(1, 3, 2)
hourly_avg = df.groupby('hr')['cnt'].mean()
sns.lineplot(x=hourly_avg.index, y=hourly_avg.values)
plt.title("Média de Aluguéis por Hora")
plt.xlabel("Hora do Dia")
plt.ylabel("Média de Bicicletas")
plt.grid(True)

# Heatmap de correlação
plt.subplot(1, 3, 3)
sns.heatmap(df.corr(numeric_only=True), cmap='coolwarm', center=0)
plt.title("Mapa de Correlação")

plt.tight_layout()
plt.show()

print("\n" + "="*60)
print("INSIGHTS RÁPIDOS")
print("="*60)
print("""
• A variável 'cnt' representa o total de aluguéis por hora.
• Espera-se forte correlação com:
    - registered
    - temp / atemp
    - hr (efeito horário)
• Variáveis climáticas influenciam significativamente o volume.
• Distribuição de 'cnt' geralmente apresenta assimetria positiva.
""")


In [ ]:
# 2. FEATURE ENGINEERING

# Codificação Cíclica da Hora
df['hr_sin'] = np.sin(2 * np.pi * df['hr'] / 24)
df['hr_cos'] = np.cos(2 * np.pi * df['hr'] / 24)

# Interações importantes
df['temp_hr'] = df['temp'] * df['hr_sin']
df['humidity_temp'] = df['hum'] * df['temp']

# Target
y = df['cnt'].values.reshape(-1, 1)

# Removendo colunas que causam data leakage
X_raw = df.drop(columns=['cnt', 'casual', 'registered', 'instant', 'dteday', 'hr'])

# One-hot nas categóricas (sem hr agora)
cat_cols = ['season', 'mnth', 'weekday', 'weathersit']
X_encoded = pd.get_dummies(X_raw, columns=cat_cols, drop_first=True)

X = X_encoded.values


In [ ]:
# 3. SPLIT

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [ ]:
# 4. ESCALONAMENTO

scaler_X = StandardScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

scaler_y = StandardScaler()
y_train_scaled = scaler_y.fit_transform(y_train)
y_test_scaled = scaler_y.transform(y_test)


In [ ]:
# 5. TENSORES

X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_scaled, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test_scaled, dtype=torch.float32)


In [ ]:
# 6. MODELO MLP REGULARIZADO

class MLPRegressor(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Dropout(0.3),

            nn.Linear(128, 64),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Dropout(0.2),

            nn.Linear(64, 1)
        )

    def forward(self, x):
        return self.model(x)

input_dim = X_train_tensor.shape[1]
model = MLPRegressor(input_dim)

criterion = nn.HuberLoss(delta=1.0)
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)


In [ ]:
# 7. TREINAMENTO COM EARLY STOPPING

epochs = 1000
patience = 50
best_loss = float('inf')
counter = 0

train_losses = []
val_losses = []

for epoch in range(epochs):

    # ---- Treino ----
    model.train()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # ---- Validação ----
    model.eval()
    with torch.no_grad():
        val_pred = model(X_test_tensor)
        val_loss = criterion(val_pred, y_test_tensor)

    train_losses.append(loss.item())
    val_losses.append(val_loss.item())

    # Early stopping
    if val_loss < best_loss:
        best_loss = val_loss
        counter = 0
        torch.save(model.state_dict(), "best_model.pth")
    else:
        counter += 1

    if counter >= patience:
        print(f"Early stopping ativado na época {epoch+1}")
        break

    if (epoch+1) % 100 == 0:
        print(f"Epoch [{epoch+1}], Train Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}")

# Carrega melhor modelo
model.load_state_dict(torch.load("best_model.pth"))


In [ ]:
# 8. AVALIAÇÃO FINAL

model.eval()
with torch.no_grad():
    predictions_scaled = model(X_test_tensor)

    predictions_real = scaler_y.inverse_transform(predictions_scaled.numpy())
    y_test_real = scaler_y.inverse_transform(y_test_tensor.numpy())

    mse = np.mean((predictions_real - y_test_real) ** 2)
    mae = np.mean(np.abs(predictions_real - y_test_real))
    r2 = r2_score(y_test_real, predictions_real)

print("\n" + "="*50)
print("RESULTADOS FINAIS OTIMIZADOS")
print("="*50)
print(f"MAE : {mae:.0f} bicicletas/hora")
print(f"MSE : {mse:.0f}")
print(f"R²  : {r2:.4f} ({r2*100:.2f}%)")
print("="*50)


In [ ]:
# 9. GRÁFICOS

fig, axes = plt.subplots(1, 3, figsize=(20, 6))

# Curva de Loss
axes[0].plot(train_losses, label='Train Loss')
axes[0].plot(val_losses, label='Validation Loss')
axes[0].set_title("Curva de Treinamento")
axes[0].legend()
axes[0].grid()

# Real vs Predito
axes[1].scatter(y_test_real, predictions_real, alpha=0.3)
min_val = min(np.min(y_test_real), np.min(predictions_real))
max_val = max(np.max(y_test_real), np.max(predictions_real))
axes[1].plot([min_val, max_val], [min_val, max_val], color='red')
axes[1].set_title("Real vs Predito")
axes[1].grid()

# Resíduos
residuos = y_test_real - predictions_real
sns.histplot(residuos, bins=50, kde=True, ax=axes[2])
axes[2].set_title("Distribuição dos Resíduos")
axes[2].grid()

plt.tight_layout()
plt.show()
